<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/hERG-Meta-learning-Convolution-Neural-Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [28]:
df = pd.read_csv("/content/hERG_IC50_nM_new.csv")
df

,assay_chembl_id,assay_description,assay_type,canonical_smiles,document_journal,molecule_chembl_id,Value,IC50_uM,Activity,hERG
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,T,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,14.0,0.140,No,0
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,32.2,0.322,No,0
2,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,J Med Chem,CHEMBL6966,143.0,1.430,No,0
3,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,F,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,J Med Chem,CHEMBL1107,196.0,1.960,No,0
4,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,J Med Chem,CHEMBL998,173.0,1.730,No,0
...,...,...,...,...,...,...,...,...,...,...
2963,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,J Med Chem,CHEMBL5081517,44000.0,440.000,Yes,1
2964,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,J Med Chem,CHEMBL5072442,33000.0,330.000,Yes,1
2965,CHEMBL5049389,Inhibition of hERG,T,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,J Med Chem,CHEMBL5093378,670.0,6.700,No,0
2966,CHEMBL5050750,Inhibition of human ERG,T,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,ACS Med Chem Lett,CHEMBL5070887,14.0,0.140,No,0


In [70]:
!pip install rdkit --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
!pip uninstall rdkit
!pip install rdkit

Found existing installation: rdkit 2023.3.1
Uninstalling rdkit-2023.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/rdkit-2023.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitAbbreviations-9a0393d8.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitAlignment-7a2a84aa.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitAvalonLib-94315007.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitCIPLabeler-2f489175.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitCatalogs-2691a182.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitChemReactions-5bcaae80.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitChemTransforms-06583482.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitChemicalFeatures-88798102.so.1
    /usr/local/lib/python3.10/dist-packages/rdkit.libs/libRDKitDataStructs-139717ff.so.1
    /usr/local/lib/python3.10/dist-

In [29]:
from rdkit.Chem import Pharm2D
from rdkit.Chem import Descriptors
from rdkit.Chem.AtomPairs import Pairs

In [30]:
# define the fingerprints and their respective parameters
fingerprints = {
    'PubChem': (AllChem.GetMorganFingerprintAsBitVect, [2, 1024]),
    'ECFP6': (AllChem.GetMorganFingerprintAsBitVect, [3, 2048]),
    'MACCS': (Chem.rdMolDescriptors.GetMACCSKeysFingerprint, []),
    'AP2D': (AllChem.GetHashedAtomPairFingerprintAsBitVect, [2048]),
    'Circle': (AllChem.GetMorganFingerprintAsBitVect, [3, 2048]),
    'CKD': (Chem.rdMolDescriptors._CalcCrippenContribs, []),
    'CKDGraph': (Chem.rdmolops.GetAdjacencyMatrix, []),
}

In [31]:
# Prepare the input data for the machine learning models
X_fps = {}
for fp_name, (fp_func, fp_args) in fingerprints.items():
    fp_module = __import__(fp_func.__module__, fromlist=[fp_func.__name__])
    X_fps[fp_name] = np.asarray([getattr(fp_module, fp_func.__name__)(Chem.MolFromSmiles(smi), *fp_args) for smi in df['canonical_smiles']])


<ipython-input-31-53cdfdfae168>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_fps[fp_name] = np.asarray([getattr(fp_module, fp_func.__name__)(Chem.MolFromSmiles(smi), *fp_args) for smi in df['canonical_smiles']])


In [32]:
# initialize the models
models = {
    'RF': RandomForestClassifier(n_estimators=100),
    'ADA': AdaBoostClassifier(),
    'LGBM': LGBMClassifier(),
    #'PLS': PLSRegression(n_components=2),
    'MLP': MLPClassifier(),
    'NB': GaussianNB(),
    'DT': DecisionTreeClassifier(),
    'ET': ExtraTreesClassifier(n_estimators=100),
    'XGB': XGBClassifier(),
    'KNN': KNeighborsClassifier(),
    #'LR': LogisticRegression(),
    'SVM': SVC(kernel='linear', probability=True),
    'RBF_SVM': SVC(kernel='rbf', probability=True),
}

In [35]:
# train and evaluate the models
results = []
for fingerprint in fingerprints:
    for algorithm in models:
        # get the model
        model = models[algorithm]

        # get the fingerprint
        X = X_fps[fingerprint]

        # get the labels
        y = df['hERG'].values

        # split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # train the model
        model.fit(X_train, y_train)

        # make predictions on the testing set
        y_pred = model.predict(X_test)

        # calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # append the results to the list
        results.append({'fingerprint': fingerprint, 'algorithm': algorithm, 'accuracy': accuracy})


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


ValueError: ignored

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim